In [20]:
import requests

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)

if response.status_code == 200:
    text = response.text  # Get the webpage content as text
    with open("downloaded_text.txt", "w", encoding="utf-8") as file:
        file.write(text)
    print("Text downloaded successfully!")
else:
    print("Failed to retrieve the page.")


Text downloaded successfully!


In [1]:
# read it in to inspect it
with open('downloaded_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[x] for x in s]
decode = lambda s: ''.join([itos[x] for x in s])
print(encode('hamza'))
print(decode(encode('hamza')))

[46, 39, 51, 64, 39]
hamza


In [6]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type())
data[:100]

torch.Size([1115394]) torch.LongTensor


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [7]:
n = int(0.9*(len(data)))
train_data = data[:n]
val_data = data[n:]

In [10]:
torch.manual_seed(1337)

def get_batch(get_split):
    data = train_data if get_split == 'train' else val_data
    n = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in n])
    y = torch.stack([data[i+1:i+block_size+1] for i in n])
    return x, y

train, test = get_batch('train')
print('input data')
print(train)
print('out data')
print(test)

input data
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46],
        [43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39],
        [50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32],
        [ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40],
        [52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1, 51, 53],
        [39, 49, 12,  1

In [11]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [242]:
batch_size = 32
block_size = 128
train_iter = 1000
val_iter = 50
learning_rate = 3e-4
n_embd = 120
num_head = 6
num_layer = 4
dropout = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [213]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = k @ q.transpose(-2, -1)* C**0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

In [214]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [215]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

In [216]:
class Block(nn.Module):
    def __init__(self, n_embd, num_head):
        super().__init__()
        head_size = n_embd // num_head
        self.sa_head = MultiHeadAttention(num_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.layer_norm1 = nn.LayerNorm(n_embd)
        self.layer_norm2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa_head(self.layer_norm1(x))
        x = x + self.ffwd(self.layer_norm2(x))
        return x


In [217]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # (32, 8, 32)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) #(8, 32)
        self.blocks = nn.Sequential(*[Block(n_embd, num_head) for _ in range(num_layer)])
        self.lnf = nn.LayerNorm(n_embd) # final layer normalization
        self.lm_head = nn.Linear(n_embd, vocab_size) # (32, 8, 32) @ (32,65)
        

    def forward(self, idx, target= None):
        B, T = idx.shape
        tok_embd = self.token_embedding_table(idx) # (65,32)
        pos_embd = self.position_embedding_table(torch.arange(T)) #(8, 32)
        x = tok_embd + pos_embd # (32, 8, 32)
        x = self.blocks(x)
        x = self.lnf(x) # final layer normalization
        logits = self.lm_head(x) # (32,65)
        if target is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)

        return logits, loss#, tok_embd.shape, pos_embd.shape, x.shape, logits.shape

    def Generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=-1)
        return idx

train, test = get_batch('train')
M = BigramLanguageModel()
M = M.to(device)
logits, loss= M(train,test)#, t_shape, p_shape, x_shape, l_shape = M(train,test)
print(logits.shape)
print(loss)

print(decode(M.Generate(idx = torch.zeros((1, 1), dtype=torch.long, device = device), max_new_tokens=100)[0].tolist()))

torch.Size([4096, 65])
tensor(4.4069, grad_fn=<NllLossBackward0>)

.;xiizJ;CWHcyubVxcjGTDaJC$BaSdkQkQqPSifGuZb:OX3bYYnA&&aDHG tZtVCVqFCdQrEQfs tAJZgYv;vOA:KTdPyvqnsnfk


In [218]:
optimizer = torch.optim.Adam(M.parameters(), lr=learning_rate)

In [249]:
tl = []
for _ in range(train_iter):
    x , y = get_batch('train')
    logits ,loss = M.forward(x, y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    tl.append(loss.item())
    

print('train loss is:',sum(tl) / len(tl))

train loss is: 1.7876832123994828


In [250]:
vl = []
x , y = get_batch('val')
for _ in range(val_iter):
    x , y = get_batch('train')
    logits ,loss = M.forward(x, y)
    vl.append(loss.item())

print('validation loss is:',sum(vl) / len(vl))

validation loss is: 1.7717492437362672


In [251]:
print(decode(M.Generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


Grapt oll-shall hants more in fraid
Shrang with of on, I ceath, a wrobortake.

LANGS:
Ved spead no.

CAPULIUS:
Wo'ell in a besin the star,
In that womaddeofulon thou eng to selve 'tch, bay
my ong soulded quity ina
And from wire he hand; honows, was Mans;
Mine and a have your's by by urselicloat oplems:
Secold, as Hast daad our sain fring,
If ambleds a be the fin: to handy comillain a day.
Firsures!
It Buchillorainature my him, Last, ord! Comearia atiman, the diss lave s;
Heccoe hows in bit hand, they strupts arlonds
And twato aniour Clo littnumblers in the beself
Bowast nase lew him Byady beleseo;
Thands a 'my now will a for the genge rother: 
All brormsafe swes do ball:
This in apent, word, insly, a I scakes,
I draudy
frien and this ever reat a fongarimpt man.

FORIAR O:
Thy most: spas thou woe finds,
Ine anat will for thou heavert's you sus,
That lay low him he they-too of of is by and beedo: yor bumbt,
pon thoughimse, I day, an be but were?

Surn Ans or I smen:
You ereforth rave is

In [172]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [173]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [182]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [181]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
xbow2

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [192]:
wei = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(wei==0, float('-inf'))
wei = F.softmax(wei, 1)
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True

In [193]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])